# Chains in LangChain

Typically, a chain integrates an LLM with a prompt, forming modules that can execute a sequence of operations on our text or other datasets. These chains are designed to process multiple inputs simultaneously. The most common LLM providers are OpenAI, Cohere, Bloom, Huggingface etc.

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [0]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import openai
#from langchain.llms import OpenAI
from langchain.llms import AzureOpenAI

#from langchain.chat_models import ChatOpenAI
from langchain.chat_models import AzureChatOpenAI

from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.docstore.document import Document

from langchain.chains import LLMChain
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains.summarize import load_summarize_chain
from langchain.chains import ConversationChain
from langchain.chains import SimpleSequentialChain
from langchain.chains import SequentialChain
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

from langchain.memory import ConversationBufferMemory

import tiktoken

#### Loading the ´gpt-35-turbo´ model

In [0]:
openai.api_type = "azure"
openai.api_base = "https://rg-rbi-aa-aitest-dsacademy.openai.azure.com/"
#openai.api_base = "https://chatgpt-summarization.openai.azure.com/"
openai.api_key = os.environ["OPENAI_API_KEY"]

openai_model_name = "gpt-35-turbo"
openai_deploy_name = "model-gpt-35-turbo"
openai.api_version = "2023-07-01-preview"

In [0]:
llm = AzureChatOpenAI(openai_api_base=openai.api_base,
                      openai_api_version=openai.api_version,
                      deployment_name=openai_deploy_name,
                      openai_api_key=os.environ["OPENAI_API_KEY"],
                      openai_api_type=openai.api_type,
                      temperature=0.9
                      )


llm

#### Loading data for using in the chains

In [0]:
df = pd.read_csv('./Data/Data.csv')

In [0]:
df.head()

## LLMChain

An LLMChain is a basic but the most commonly used type of chain. It consists of a PromptTemplate, an Open AI model (an LLM or a ChatModel), and an optional output parser. LLM chain takes multiple input variables and uses the PromptTemplate to format them into a prompt. It passes the prompt to the model. Finally, it uses the OutputParser (if provided) to parse the output of the LLM into a final format. In this simple chain, we are just chaining the llm and the prompt

In [0]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

The variables of the prompted are passed when we call chain.run (only one variable)

In [0]:
chain = LLMChain(llm=llm, prompt=prompt)
product = "Queen Size Sheet Set"
chain.run(product)


## SequentialChain
A sequential chain combines multiple chains where the output of one chain is the input of the next chain. It runs a sequence of chains one after another.  
There are 2 types of sequential chains:

+ SimpleSequentialChain — single input/output
+ SequentialChain — multiple inputs/outputs

## SimpleSequentialChain  
This is the simplest form of a sequential chain, where each step has a singular input/output, and the output of one step is the input to the next. This works well when we have subchains that expect only one input and return only one output.  

First prompt template and chain

In [0]:
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe \
    a company that makes {product}?"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt)

Second prompt template and chain

In [0]:
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following \
    company:{company_name}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt)

Joining the two chains:

In [0]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [0]:
overall_simple_chain.run(product)

## SequentialChain  
Not all sequential chains involve only passing a single string as an input and getting a single string as an output.   
More complex chains involve multiple inputs and multiple final outputs.  

![Sequential Chain](https://miro.medium.com/v2/resize:fit:828/format:webp/1*hdx24fJuQwWm1fT-ULGQhg.jpeg)

Here, the naming of input/output variables is very important in this chain.

First prompt template and chain

In [0]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to english:"
    "\n\n{Review}"
)
# chain 1: input= Review and output = English_Review
chain_one = LLMChain(llm=llm, 
                     prompt=first_prompt, 
                     output_key="English_Review"
                    )


Second prompt template and chain

In [0]:
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following review in 1 sentence:"
    "\n\n{English_Review}"
)
# chain 2: input= English_Review and output= summary
chain_two = LLMChain(llm=llm, 
                     prompt=second_prompt, 
                     output_key="summary"
                    )


Third prompt template and chain

In [0]:
# prompt template 3: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review:\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, 
                       prompt=third_prompt,
                       output_key="language"
                      )

Fourth prompt template and chain

In [0]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(llm=llm, 
                      prompt=fourth_prompt,
                      output_key="followup_message"
                     )

Overall_chain: input = Review and output = English_Review, summary, followup_message

In [0]:

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary","followup_message"],
    verbose=True
)

In [0]:
review = df.Review[5]
overall_chain(review)

## Router Chain

RouterChain is used for complicated tasks. For example, a pretty common but basic operation is to route an input to a chain depending on what exactly that input is. If we have multiple subchains, each of which is specialized for a particular type of input, we could have a router chain that decides which subchain to pass the input to. For example, we can route between multiple subchains depending on each specialized for a particular type of input.

![](https://miro.medium.com/v2/resize:fit:900/format:webp/1*0TDSAfaL2Q46TnFWkQTagg.jpeg)

See the templates in the example below:

In [0]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""

computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

And the information for every template, to allow the LLM to decide:

In [0]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

Now we will create all possible destination chains.

In [0]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

And also a generic chain if none of the topics are related to the question

In [0]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

Let's create a generic router template to decide which prompt to use afterwards

In [0]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

Adjusting the pieces together to create the router chain: 

In [0]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(template=router_template,
                               input_variables=["input"],
                               output_parser=RouterOutputParser(),
                               )

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [0]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [0]:
chain.run("What is black body radiation?")

In [0]:
chain.run("what is 2 + 2")

In [0]:
chain.run("Why does every cell in our body contain DNA?")